In [1]:
import duckdb as ddb
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec

data = ddb.read_parquet("all_reviews/all_reviews.parquet")

In [ ]:
# columns and types
for x, item in enumerate(data.columns):
    print(str(item) + ":  " + str(data.dtypes[x]))

game:  VARCHAR
author_steamid:  BIGINT
author_num_games_owned:  INTEGER
author_num_reviews:  INTEGER
author_playtime_forever:  INTEGER
author_playtime_last_two_weeks:  INTEGER
author_playtime_at_review:  BIGINT
author_last_played:  INTEGER
language:  VARCHAR
review:  VARCHAR
timestamp_created:  INTEGER
timestamp_updated:  INTEGER
voted_up:  INTEGER
votes_up:  INTEGER
votes_funny:  INTEGER
weighted_vote_score:  FLOAT
comment_count:  INTEGER
steam_purchase:  INTEGER
received_for_free:  INTEGER
written_during_early_access:  INTEGER


In [ ]:
# print(data.describe())


['game', 'author_steamid', 'author_num_games_owned', 'author_num_reviews', 'author_playtime_forever', 'author_playtime_last_two_weeks', 'author_playtime_at_review', 'author_last_played', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'voted_up', 'votes_up', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access']
┌─────────┬─────────────────────────────┬───────────────────────┬────────────────────────┬────────────────────┬─────────────────────────┬────────────────────────────────┬───────────────────────────┬────────────────────┬────────────┬───────────────────────────────────────────────┬────────────────────┬────────────────────┬───────────────────┬────────────────────┬────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────────────┐
│  aggr   │            game             │    author_steamid     │ author_num_games_owned │ author

In [ ]:
# review scores in early access vs not
#   which games had a high jump from early access / drop in review score after full release

# plan: 
#   gather the top 3 games the most amount of reviews
#   find out the percentage of positive vs negative reviews
#   then find out the percentages from those who got the games in early access vs. full release
#   analyze which games had which results, and do some further research on why this may be
#       this could be finding the reviews with the highest helpfulness or most upvotes
#       this could also include doing some outside reseach on reddit / internet

most_reviewed_games = ddb.sql(f"""
    WITH ranked_games AS (
                        
                              )
""").fetchdf()

print(most_reviewed_games)

   user_id     avg_dist
0  5160944  1379.085118
1  6031349  1341.881405
2   102799  1334.697030
3  7285664  1327.378560
4  6059850  1320.024138
5  3478514  1310.574134
6  6414017  1301.147320
7  3342586  1296.243675
8  8520573  1295.947087
9  6493276  1289.837205


In [ ]:
# - review scores from those who got the game for free vs those who had to pay for it
#     - is there an average of a higher rating for free? for what games?
avg_distances = ddb.sql(f"""
    
""").fetchdf()

print(avg_distances.head(10))